
# TradingView Strategy Backtest Playground

Notebook ini memuat pipeline lengkap untuk menguji sinyal strategi yang diekspor dari TradingView.
Ia memandu mulai dari memuat data CSV, mengadaptasikannya ke format QF-Lib, menjalankan backtest,
hingga analisis trade kalah dan eksperimen optimasi parameter tambahan.



> **Struktur notebook**
>
> 1. Parameter input & pemuatan data
> 2. Adaptasi data ke QF-Lib
> 3. Strategi berbasis sinyal
> 4. Menjalankan backtest & mengekstrak hasil
> 5. Visualisasi
> 6. Analisis trade kalah & investigasi
> 7. Eksplor optimasi parameter
> 8. Dokumentasi & reusable structure


In [ ]:

from __future__ import annotations

from dataclasses import dataclass, field
from pathlib import Path
from typing import Dict, Iterable, List, Optional, Tuple
import itertools
import math
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-darkgrid")

PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.qflib_metrics import qflib_metrics_from_returns
from src.qflib_adapters import to_qfdataframe, to_qfseries

try:
    from qf_lib.common.enums.price_field import PriceField
    from qf_lib.analysis.timeseries_analysis.timeseries_analysis import TimeseriesAnalysis  # noqa: F401
    from qf_lib.containers.dataframe.qf_dataframe import QFDataFrame
except ImportError as exc:  # pragma: no cover - dependency guard
    raise ImportError(
        "qf-lib harus terinstal. Pastikan `pip install -r requirements.txt` sudah dijalankan."
    ) from exc


## 1. Parameter input & pemuatan data

In [ ]:

import re


def sanitise_column_name(name: str) -> str:
    """Konversi nama kolom ke format snake_case yang stabil."""
    replacements = {"+": " plus ", "-": " minus ", "@": " at ", "%": " pct "}
    for old, new in replacements.items():
        name = name.replace(old, new)
    name = re.sub(r"[^0-9a-zA-Z]+", "_", name)
    name = re.sub(r"_+", "_", name)
    return name.strip("_").lower()


def sanitise_columns(columns: Iterable[str]) -> Tuple[List[str], Dict[str, str]]:
    """Berikan daftar nama kolom unik serta mapping sanitised -> original."""
    seen: Dict[str, int] = {}
    sanitised: List[str] = []
    mapping: Dict[str, str] = {}

    for original in columns:
        base = sanitise_column_name(original)
        count = seen.get(base, 0)
        if count > 0:
            candidate = f"{base}_{count + 1}"
        else:
            candidate = base
        seen[base] = count + 1
        sanitised.append(candidate)
        mapping[candidate] = original

    return sanitised, mapping


def load_strategy_csv(
    path: Path,
    time_column: str,
    tz_localize: Optional[str] = None,
    tz_convert: Optional[str] = None,
) -> Tuple[pd.DataFrame, Dict[str, str]]:
    """Muat CSV strategi TradingView dan kembalikan DataFrame bernomor float dengan index datetime."""
    if not path.exists():
        raise FileNotFoundError(f"File tidak ditemukan: {path}")

    df = pd.read_csv(path)
    if time_column not in df.columns:
        raise KeyError(f"Kolom waktu '{time_column}' tidak ada di file {path.name}")

    time_values = df.pop(time_column)
    if np.issubdtype(time_values.dtype, np.number):
        index = pd.to_datetime(time_values, unit="s", utc=True)
    else:
        index = pd.to_datetime(time_values, utc=True, infer_datetime_format=True)

    if tz_localize is not None:
        index = index.tz_localize(tz_localize)
    if tz_convert is not None:
        index = index.tz_convert(tz_convert)

    index = index.tz_convert(None)

    sanitised_names, mapping = sanitise_columns(df.columns)
    df.columns = sanitised_names
    df.index = index
    df = df.sort_index()

    numeric_df = df.apply(pd.to_numeric, errors="coerce")
    return numeric_df, mapping


In [ ]:

DATA_FILE = "OKX_BTCUSDT, 1D.csv"
TIME_COLUMN = "time"
PRICE_COLUMN = "close"
ASSUME_SAME_BAR_EXECUTION = True

signal_columns = {
    "long_entry": sanitise_column_name("Reversal Up +"),
    "long_exit": sanitise_column_name("Reversal Down -"),
    "short_entry": sanitise_column_name("Reversal Down +"),
    "short_exit": sanitise_column_name("Reversal Up -"),
}

filters_template = {
    "min_money_flow_long": None,
    "max_money_flow_short": None,
    "min_confluence_value": None,
    "require_ema_trend_confirmation": False,
}

context_feature_candidates = [
    "ema",
    "money_flow",
    "confluence_meter_value",
    "atr",
    "macd",
    "signal",
    "histogram",
]

DATA_PATH = PROJECT_ROOT / "data" / DATA_FILE
raw_df, column_lookup = load_strategy_csv(DATA_PATH, TIME_COLUMN)

available_context_columns = [col for col in context_feature_candidates if col in raw_df.columns]

print(f"Total bar data: {len(raw_df):,}")
print(f"Rentang tanggal: {raw_df.index.min()} → {raw_df.index.max()}")
print(f"Kolom harga utama tersedia: {[c for c in ['open','high','low','close'] if c in raw_df.columns]}")


In [ ]:

column_overview = (
    pd.DataFrame(
        {
            "sanitised": list(column_lookup.keys()),
            "original": list(column_lookup.values()),
        }
    )
    .sort_values("sanitised")
    .reset_index(drop=True)
)
column_overview.head(20)


In [ ]:

if PRICE_COLUMN not in raw_df.columns:
    raise KeyError(f"Kolom harga '{PRICE_COLUMN}' tidak ditemukan dalam data yang disanitasi")

missing_signals = [alias for alias in signal_columns.values() if alias not in raw_df.columns]
if missing_signals:
    raise KeyError(
        "Kolom sinyal berikut tidak ditemukan. Periksa parameter `signal_columns`: "
        + ", ".join(missing_signals)
    )

signals_preview = raw_df[list(dict.fromkeys(signal_columns.values()))].copy()
signals_preview.head()


In [ ]:

df = raw_df.copy()

df["ema_slope"] = df["ema"].diff() if "ema" in df.columns else np.nan
if "money_flow" in df.columns:
    df["money_flow_z"] = pd.qcut(df["money_flow"], q=5, duplicates="drop")
if "confluence_meter_value" in df.columns:
    df["confluence_meter_rank"] = pd.qcut(df["confluence_meter_value"], q=5, duplicates="drop")

context_columns = [*available_context_columns]
if "ema_slope" in df.columns:
    context_columns.append("ema_slope")

print(f"Context features yang digunakan pada log trade: {context_columns}")


## 2. Adaptasi data ke QF-Lib

In [ ]:

price_fields = {
    PriceField.Open: df["open"] if "open" in df.columns else pd.Series(np.nan, index=df.index),
    PriceField.High: df["high"] if "high" in df.columns else pd.Series(np.nan, index=df.index),
    PriceField.Low: df["low"] if "low" in df.columns else pd.Series(np.nan, index=df.index),
    PriceField.Close: df[PRICE_COLUMN],
}

price_qf = QFDataFrame(pd.DataFrame(price_fields))
signal_qf = to_qfdataframe(df[list(dict.fromkeys(signal_columns.values()))])

price_qf.head()


## 3. Strategi berbasis sinyal

In [ ]:

@dataclass
class TradeRecord:
    trade_id: int
    direction: str
    entry_time: pd.Timestamp
    exit_time: pd.Timestamp
    entry_price: float
    exit_price: float
    pnl_pct: float
    pnl_currency: float
    bars_held: int
    exit_reason: str
    entry_context: Dict[str, float] = field(default_factory=dict)
    exit_context: Dict[str, float] = field(default_factory=dict)


class ExcelSignalStrategy:
    """Bangun posisi berdasarkan sinyal yang sudah dihitung di CSV TradingView."""

    def __init__(
        self,
        data: pd.DataFrame,
        price_column: str,
        signal_columns: Dict[str, str],
        filters: Dict[str, Optional[float]],
        context_columns: Optional[List[str]] = None,
        assume_same_bar_execution: bool = True,
    ) -> None:
        self.data = data
        self.price_column = price_column
        self.signal_columns = signal_columns
        self.filters = filters
        self.context_columns = context_columns or []
        self.assume_same_bar_execution = assume_same_bar_execution

        missing = [col for col in signal_columns.values() if col not in data.columns]
        if missing:
            raise KeyError(f"Kolom sinyal tidak ditemukan dalam data: {missing}")

        if price_column not in data.columns:
            raise KeyError(f"Kolom harga '{price_column}' tidak tersedia")

        self._signal_frame = {
            name: self._to_boolean_series(self.data[column])
            for name, column in self.signal_columns.items()
        }

    @staticmethod
    def _to_boolean_series(series: pd.Series) -> pd.Series:
        if series.dtype == bool:
            return series.fillna(False)
        numeric = pd.to_numeric(series, errors="coerce")
        if numeric.notna().any():
            return numeric.fillna(0.0).astype(float).abs() > 0.0
        return series.astype(str).str.strip().ne("")

    def _extract_context(self, row: pd.Series) -> Dict[str, float]:
        context = {}
        for col in self.context_columns:
            context[col] = float(row.get(col, np.nan)) if pd.notna(row.get(col, np.nan)) else np.nan
        return context

    def _passes_long_filters(self, row: pd.Series) -> bool:
        if self.filters.get("min_money_flow_long") is not None and "money_flow" in row.index:
            if row["money_flow"] < float(self.filters["min_money_flow_long"]):
                return False
        if self.filters.get("min_confluence_value") is not None and "confluence_meter_value" in row.index:
            if row["confluence_meter_value"] < float(self.filters["min_confluence_value"]):
                return False
        if self.filters.get("require_ema_trend_confirmation") and "ema_slope" in row.index:
            if row["ema_slope"] < 0:
                return False
        return True

    def _passes_short_filters(self, row: pd.Series) -> bool:
        if self.filters.get("max_money_flow_short") is not None and "money_flow" in row.index:
            if row["money_flow"] > float(self.filters["max_money_flow_short"]):
                return False
        if self.filters.get("min_confluence_value") is not None and "confluence_meter_value" in row.index:
            if row["confluence_meter_value"] < float(self.filters["min_confluence_value"]):
                return False
        if self.filters.get("require_ema_trend_confirmation") and "ema_slope" in row.index:
            if row["ema_slope"] > 0:
                return False
        return True

    def run(self) -> Tuple[pd.Series, pd.DataFrame]:
        index = self.data.index
        position_values = np.zeros(len(index), dtype=float)
        trades: List[TradeRecord] = []

        current_pos = 0
        entry_idx: Optional[int] = None
        entry_price: Optional[float] = None
        entry_row: Optional[pd.Series] = None
        entry_direction: Optional[str] = None

        long_entry_series = self._signal_frame.get("long_entry")
        long_exit_series = self._signal_frame.get("long_exit")
        short_entry_series = self._signal_frame.get("short_entry")
        short_exit_series = self._signal_frame.get("short_exit")

        for i, (timestamp, row) in enumerate(self.data.iterrows()):
            price = float(row[self.price_column])
            long_entry = bool(long_entry_series.iloc[i]) if long_entry_series is not None else False
            long_exit = bool(long_exit_series.iloc[i]) if long_exit_series is not None else False
            short_entry = bool(short_entry_series.iloc[i]) if short_entry_series is not None else False
            short_exit = bool(short_exit_series.iloc[i]) if short_exit_series is not None else False

            exit_trade = False
            exit_reason = ""

            if current_pos > 0:
                if long_exit:
                    exit_trade = True
                    exit_reason = "long_exit_signal"
                elif short_entry:
                    exit_trade = True
                    exit_reason = "short_reversal"
            elif current_pos < 0:
                if short_exit:
                    exit_trade = True
                    exit_reason = "short_exit_signal"
                elif long_entry:
                    exit_trade = True
                    exit_reason = "long_reversal"

            if exit_trade and entry_idx is not None and entry_price is not None and entry_direction is not None:
                pnl_pct = (price / entry_price - 1.0)
                if entry_direction == "Short":
                    pnl_pct = -pnl_pct
                pnl_currency = pnl_pct * entry_price
                bars_held = i - entry_idx
                trade = TradeRecord(
                    trade_id=len(trades) + 1,
                    direction=entry_direction,
                    entry_time=index[entry_idx],
                    exit_time=timestamp,
                    entry_price=entry_price,
                    exit_price=price,
                    pnl_pct=pnl_pct,
                    pnl_currency=pnl_currency,
                    bars_held=bars_held,
                    exit_reason=exit_reason,
                    entry_context=self._extract_context(entry_row) if entry_row is not None else {},
                    exit_context=self._extract_context(row),
                )
                trades.append(trade)
                current_pos = 0
                entry_idx = None
                entry_price = None
                entry_row = None
                entry_direction = None

            if current_pos == 0:
                if long_entry and self._passes_long_filters(row):
                    current_pos = 1
                    entry_idx = i
                    entry_price = price
                    entry_row = row
                    entry_direction = "Long"
                elif short_entry and self._passes_short_filters(row):
                    current_pos = -1
                    entry_idx = i
                    entry_price = price
                    entry_row = row
                    entry_direction = "Short"

            position_values[i] = current_pos

        if current_pos != 0 and entry_idx is not None and entry_price is not None and entry_direction is not None:
            timestamp = index[-1]
            price = float(self.data.iloc[-1][self.price_column])
            pnl_pct = (price / entry_price - 1.0)
            if entry_direction == "Short":
                pnl_pct = -pnl_pct
            pnl_currency = pnl_pct * entry_price
            bars_held = len(index) - entry_idx - 1
            trade = TradeRecord(
                trade_id=len(trades) + 1,
                direction=entry_direction,
                entry_time=index[entry_idx],
                exit_time=timestamp,
                entry_price=entry_price,
                exit_price=price,
                pnl_pct=pnl_pct,
                pnl_currency=pnl_currency,
                bars_held=bars_held,
                exit_reason="forced_exit_at_end",
                entry_context=self._extract_context(entry_row) if entry_row is not None else {},
                exit_context=self._extract_context(self.data.iloc[-1]),
            )
            trades.append(trade)

        positions = pd.Series(position_values, index=index, name="position")

        trade_records = []
        for trade in trades:
            record = {
                "trade_id": trade.trade_id,
                "direction": trade.direction,
                "entry_time": trade.entry_time,
                "exit_time": trade.exit_time,
                "entry_price": trade.entry_price,
                "exit_price": trade.exit_price,
                "pnl_pct": trade.pnl_pct,
                "pnl_currency": trade.pnl_currency,
                "bars_held": trade.bars_held,
                "exit_reason": trade.exit_reason,
            }
            for key, value in (trade.entry_context or {}).items():
                record[f"entry_{key}"] = value
            for key, value in (trade.exit_context or {}).items():
                record[f"exit_{key}"] = value
            trade_records.append(record)

        trades_df = pd.DataFrame(trade_records)
        if not trades_df.empty:
            trades_df = trades_df.sort_values("entry_time").reset_index(drop=True)

        return positions, trades_df


## 4. Menjalankan backtest & mengekstrak hasil

In [ ]:

def run_backtest_pipeline(
    data: pd.DataFrame,
    price_column: str,
    signal_columns: Dict[str, str],
    filters: Dict[str, Optional[float]],
    context_columns: Optional[List[str]] = None,
    assume_same_bar_execution: bool = True,
) -> Dict[str, object]:
    strategy = ExcelSignalStrategy(
        data=data,
        price_column=price_column,
        signal_columns=signal_columns,
        filters=filters,
        context_columns=context_columns,
        assume_same_bar_execution=assume_same_bar_execution,
    )

    positions, trades_df = strategy.run()

    asset_returns = data[price_column].pct_change().fillna(0.0)
    strategy_returns = positions.shift(1).fillna(0.0) * asset_returns
    equity_curve = (1.0 + strategy_returns).cumprod()

    results_df = pd.DataFrame(
        {
            "close": data[price_column],
            "asset_return": asset_returns,
            "position": positions,
            "strategy_return": strategy_returns,
            "equity_curve": equity_curve,
        }
    )

    rolling_max = results_df["equity_curve"].cummax()
    results_df["drawdown"] = results_df["equity_curve"] / rolling_max - 1.0

    metrics = qflib_metrics_from_returns(strategy_returns)

    trade_summary = {
        "total_trades": int(len(trades_df)),
        "long_trades": int((trades_df["direction"] == "Long").sum()) if not trades_df.empty else 0,
        "short_trades": int((trades_df["direction"] == "Short").sum()) if not trades_df.empty else 0,
        "win_rate": float((trades_df["pnl_pct"] > 0).mean()) if not trades_df.empty else np.nan,
        "avg_pnl_pct": float(trades_df["pnl_pct"].mean()) if not trades_df.empty else np.nan,
        "median_bars": float(trades_df["bars_held"].median()) if not trades_df.empty else np.nan,
    }

    return {
        "positions": positions,
        "trades": trades_df,
        "results": results_df,
        "metrics": metrics,
        "trade_summary": trade_summary,
    }


In [ ]:

base_filters = filters_template.copy()
backtest_outputs = run_backtest_pipeline(
    data=df,
    price_column=PRICE_COLUMN,
    signal_columns=signal_columns,
    filters=base_filters,
    context_columns=context_columns,
    assume_same_bar_execution=ASSUME_SAME_BAR_EXECUTION,
)

metrics_series = pd.Series(backtest_outputs["metrics"])
summary_series = pd.Series(backtest_outputs["trade_summary"])

print("=== QF-Lib Metrics ===")
display(metrics_series.to_frame("value"))

print("=== Ringkasan Trade ===")
display(summary_series.to_frame("value"))


## 5. Visualisasi

In [ ]:

results_df = backtest_outputs["results"]
trades_df = backtest_outputs["trades"]

fig, axes = plt.subplots(3, 1, figsize=(16, 18), sharex=True)

axes[0].plot(results_df.index, results_df["close"], label="Close", color="#1f77b4")
if not trades_df.empty:
    long_trades = trades_df[trades_df["direction"] == "Long"]
    short_trades = trades_df[trades_df["direction"] == "Short"]
    axes[0].scatter(long_trades["entry_time"], long_trades["entry_price"], marker="^", color="#2ca02c", s=80, label="Long Entry")
    axes[0].scatter(long_trades["exit_time"], long_trades["exit_price"], marker="v", color="#98df8a", s=80, label="Long Exit")
    axes[0].scatter(short_trades["entry_time"], short_trades["entry_price"], marker="v", color="#d62728", s=80, label="Short Entry")
    axes[0].scatter(short_trades["exit_time"], short_trades["exit_price"], marker="^", color="#ff9896", s=80, label="Short Exit")
axes[0].set_title("Harga & Sinyal Entry/Exit")
axes[0].set_ylabel("Harga")
axes[0].legend(loc="upper left")

axes[1].plot(results_df.index, results_df["equity_curve"], color="#ff7f0e", label="Equity Curve")
axes[1].fill_between(results_df.index, results_df["equity_curve"], results_df["equity_curve"].cummax(), color="#ffbb78", alpha=0.3)
axes[1].set_title("Equity Curve & Drawdown")
axes[1].set_ylabel("Equity")
axes[1].legend(loc="upper left")

axes[2].plot(results_df.index, results_df["position"], color="#9467bd")
axes[2].set_title("Posisi (1=Long, -1=Short, 0=Flat)")
axes[2].set_ylabel("Posisi")
axes[2].set_xlabel("Tanggal")

plt.tight_layout()
plt.show()

if not trades_df.empty:
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    ax[0].hist(trades_df["pnl_pct"] * 100.0, bins=20, color="#1f77b4", alpha=0.8)
    ax[0].set_title("Distribusi PnL per Trade (%)")
    ax[0].set_xlabel("PnL (%)")
    ax[0].set_ylabel("Jumlah Trade")

    ax[1].scatter(trades_df["bars_held"], trades_df["pnl_pct"] * 100.0, color="#ff7f0e", alpha=0.7)
    ax[1].set_title("Durasi vs PnL")
    ax[1].set_xlabel("Bars Held")
    ax[1].set_ylabel("PnL (%)")
    plt.tight_layout()
    plt.show()


## 6. Analisis trade kalah & investigasi

In [ ]:

trades_df = backtest_outputs["trades"]

if trades_df.empty:
    print("Tidak ada trade untuk dianalisis.")
else:
    losing_trades = trades_df[trades_df["pnl_pct"] < 0].copy()
    print(f"Total losing trades: {len(losing_trades)} dari {len(trades_df)} total trade")

    if not losing_trades.empty:
        if "entry_ema_slope" in losing_trades.columns:
            losing_trades["ema_trend_state"] = np.where(
                losing_trades["entry_ema_slope"] >= 0, "EMA Rising", "EMA Falling"
            )
        if "entry_money_flow" in losing_trades.columns:
            losing_trades["money_flow_bucket"] = pd.cut(
                losing_trades["entry_money_flow"],
                bins=[-math.inf, 40, 60, math.inf],
                labels=["<40", "40-60", ">60"],
            )
        if "entry_confluence_meter_value" in losing_trades.columns:
            losing_trades["confluence_bucket"] = pd.cut(
                losing_trades["entry_confluence_meter_value"],
                bins=[-math.inf, 40, 60, 80, math.inf],
                labels=["<40", "40-60", "60-80", ">80"],
            )

        grouping_columns = [
            col
            for col in ["direction", "ema_trend_state", "money_flow_bucket", "confluence_bucket"]
            if col in losing_trades.columns
        ]
        if grouping_columns:
            classification = (
                losing_trades.groupby(grouping_columns)
                .agg(
                    count=("trade_id", "count"),
                    avg_loss_pct=("pnl_pct", lambda x: float(x.mean() * 100.0)),
                    median_bars=("bars_held", "median"),
                )
                .sort_values("count", ascending=False)
            )
            display(classification)
        else:
            print("Kolom konteks tidak cukup untuk klasifikasi detail.")


## 7. Eksplor optimasi parameter

In [ ]:

optimization_candidates = []
money_flow_thresholds = [None, 50, 55, 60]
confluence_thresholds = [None, 55, 65, 75]
ema_confirmation_options = [False, True]

for money_flow, confluence, ema_confirm in itertools.product(
    money_flow_thresholds, confluence_thresholds, ema_confirmation_options
):
    candidate_filters = filters_template.copy()
    candidate_filters["min_money_flow_long"] = money_flow
    candidate_filters["max_money_flow_short"] = money_flow
    candidate_filters["min_confluence_value"] = confluence
    candidate_filters["require_ema_trend_confirmation"] = ema_confirm

    outputs = run_backtest_pipeline(
        data=df,
        price_column=PRICE_COLUMN,
        signal_columns=signal_columns,
        filters=candidate_filters,
        context_columns=context_columns,
        assume_same_bar_execution=ASSUME_SAME_BAR_EXECUTION,
    )

    metrics = outputs["metrics"]
    trade_summary = outputs["trade_summary"]

    optimization_candidates.append(
        {
            "min_money_flow": money_flow,
            "min_confluence": confluence,
            "ema_confirm": ema_confirm,
            "cagr": metrics.get("cagr", np.nan),
            "sharpe": metrics.get("sharpe_ratio", np.nan),
            "max_drawdown": metrics.get("max_drawdown", np.nan),
            "total_trades": trade_summary.get("total_trades", 0),
            "win_rate": trade_summary.get("win_rate", np.nan),
        }
    )

optimization_df = pd.DataFrame(optimization_candidates)
optimization_df = optimization_df.sort_values(["cagr", "sharpe"], ascending=[False, False]).reset_index(drop=True)
optimization_df.head(10)


## 8. Dokumentasi & reusable structure


### Cara menggunakan notebook ini

1. **Set parameter input** pada sel konfigurasi (nama file, kolom sinyal, filter).
2. **Jalankan blok pemuatan data** untuk memastikan kolom yang dipakai sudah benar.
3. **Tinjau adaptasi QF-Lib** guna memverifikasi harga dan sinyal siap dipakai.
4. **Jalankan backtest** dan telaah tabel metrik + ringkasan trade.
5. **Gunakan visualisasi** untuk memahami waktu entry/exit serta kinerja ekuitas.
6. **Pelajari trade yang kalah** lewat tabel klasifikasi untuk menemukan pola kelemahan.
7. **Eksplorasi optimasi** dengan memodifikasi grid filter atau menambahkan kondisi baru.
8. **Salin fungsi utilitas** (`load_strategy_csv`, `ExcelSignalStrategy`, `run_backtest_pipeline`) ke modul terpisah
   bila ingin dijadikan library reuse di luar notebook.

Notebook ini dirancang agar cukup dengan mengganti `DATA_FILE` dan `signal_columns`, Anda dapat langsung
menguji file strategi TradingView lainnya tanpa memodifikasi kode inti.
